<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/sentence_bert_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers


     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 1.1MB 13.4MB/s 
     |████████████████████████████████| 1.1MB 49.3MB/s 
     |████████████████████████████████| 890kB 46.7MB/s 
     |████████████████████████████████| 3.0MB 50.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-cp36-none-any.whl size=91190 sha256=b9eea4ae53aab8299f7aba45d8eaa2b055daeee573ffa62b5a805f200c49af87
  Stored in directory: /root/.cache/pip/wheels/4b/8f/23/7f93e91176acc7c2d651b54fa9f01e3624a47904145c4d133e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=65a3a4ab3dd52fd974a12429ce994f5ad9d1030bc74b4bc8da5bcd5b640e4a47
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSBenchmarkDataReader, InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

In [3]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [4]:
#Check if dataset exsist. If not, download and extract  it
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

100%|██████████| 392k/392k [00:02<00:00, 134kB/s]


In [5]:
# Read the dataset
model_name = 'bert-base-nli-mean-tokens'
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [6]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

2020-10-09 10:45:30 - Load pretrained SentenceTransformer: bert-base-nli-mean-tokens
2020-10-09 10:45:30 - Did not find folder bert-base-nli-mean-tokens. Assume to download model from server.
2020-10-09 10:45:30 - Downloading sentence transformer model from https://sbert.net/models/bert-base-nli-mean-tokens.zip and saving it at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens


100%|██████████| 405M/405M [00:49<00:00, 8.22MB/s]


2020-10-09 10:46:26 - Load SentenceTransformer from folder: /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens
2020-10-09 10:46:29 - Use pytorch device: cuda


In [7]:
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [8]:
train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [9]:
# Development set: Measure correlation between cosine score and gold labels
logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


2020-10-09 10:46:29 - Read STSbenchmark dev dataset


In [10]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2020-10-09 10:46:29 - Warmup-steps: 144


In [11]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)



2020-10-09 10:48:31 - Evaluation the model on sts-dev dataset after epoch 0:
2020-10-09 10:48:36 - Cosine-Similarity :	Pearson: 0.8617	Spearman: 0.8632
2020-10-09 10:48:36 - Manhattan-Distance:	Pearson: 0.8557	Spearman: 0.8593
2020-10-09 10:48:36 - Euclidean-Distance:	Pearson: 0.8560	Spearman: 0.8597
2020-10-09 10:48:36 - Dot-Product-Similarity:	Pearson: 0.8445	Spearman: 0.8440
2020-10-09 10:48:36 - Save model to output/training_stsbenchmark_continue_training-bert-base-nli-mean-tokens-2020-10-09_10-45-30



2020-10-09 10:50:25 - Evaluation the model on sts-dev dataset after epoch 1:
2020-10-09 10:50:31 - Cosine-Similarity :	Pearson: 0.8699	Spearman: 0.8716
2020-10-09 10:50:31 - Manhattan-Distance:	Pearson: 0.8607	Spearman: 0.8657
2020-10-09 10:50:31 - Euclidean-Distance:	Pearson: 0.8611	Spearman: 0.8660
2020-10-09 10:50:31 - Dot-Product-Similarity:	Pearson: 0.8550	Spearman: 0.8559
2020-10-09 10:50:31 - Save model to output/training_stsbenchmark_continue_training-bert-base-nli-mean-tokens-2020-10-09_10-45-30


KeyboardInterrupt: ignored